Weather API EDA

The data was downloaded from the [Weather open API - Open meteo](https://archive-api.open-meteo.com)
The constructed dataset contains 6 columns: Region name, quarter, average temperature, total precipitation, sunshine duration and average windspeed. 

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
weather_df = pd.read_csv(r"D:\projects\mestrado 2025\fcd\eda_test\portugal_weather_quarterly_2019_2025.csv")
weather_df.head()

region quarter  temperature_2m_mean  precipitation_sum  sunshine_duration  \
0  Norte  2019Q1             7.940000              260.8         2574408.01   
1  Norte  2019Q2            13.502198              367.8         3463017.97   
2  Norte  2019Q3            19.126087              105.7         3747818.36   
3  Norte  2019Q4            10.657609              989.3         1844647.58   
4  Norte  2020Q1             8.982418              399.6         2348701.24   

   windspeed_10m_mean  
0            9.866667  
1            9.482418  
2            8.069565  
3           10.970652  
4           10.001099

The weather data we have can be used to explore, whether the weather affects the housing prices, for example, do sunnier regions have higher property prices or does rainfall correlate with lower prices. Later on the weather dataset can be merged with the average house pricing dataset. The weather data is collected in such a way that it  chnologically corresponds to the housing prices dataset time range. Below is a list of possible questions to explore regarding the possible connection between the change in weather and house pricing.

| Question                                                                                         | What to Explore                                        |
| ------------------------------------------------------------------------------------------------ | ------------------------------------------------------ |
| Do sunnier regions have higher property prices?                                               | Compare average sunshine duration vs. average price    |
| Does rainfall correlate with lower prices or slower growth?                                   | Regression of quarterly price change vs. precipitation |
| Are milder climates linked to premium real estate?                                            | Compare temperature ranges to prices                   |
| Are coastal windy areas priced differently?                                                   | Correlate windspeed with coastal property prices       |
<!-- | How do seasonal weather patterns affect short-term rental markets (and thus property prices)? | Combine quarterly weather and Airbnb/occupancy data    | -->


In [7]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   region               175 non-null    object 
 1   quarter              175 non-null    object 
 2   temperature_2m_mean  175 non-null    float64
 3   precipitation_sum    175 non-null    float64
 4   sunshine_duration    175 non-null    float64
 5   windspeed_10m_mean   175 non-null    float64
dtypes: float64(4), object(2)
memory usage: 8.3+ KB


Fortunately, as we can see there's no missing data, so we won't need to handle the missing values. As for the columns we have 6 of them, 2 are string values and 4 are numerical. 
1. **region** indicates a region in Portugal;
2. **quarter** that corresponds to the quarter of the year in a format 'yearQ[number from 1 to 4]'. This part will be changed when the merging with house pricing will take place;
3. **temperature** is a mean temperature in Celcius in the given region and quarter;
4. **precipitation** is a sum of precipitations in the given region and quarter, given in mm;
5. **sunshine** is a sum of sunlight duration throught the quarter in a given region, given in seconds;
6. **windspeed** is a mean wind speed across the quarter in a given region, given in km/h.

In [8]:
weather_df.describe()

temperature_2m_mean  precipitation_sum  sunshine_duration  \
count           175.000000         175.000000       1.750000e+02   
mean             15.613366         245.593714       3.067421e+06   
std               4.572461         194.500520       6.782262e+05   
min               6.881111           8.800000       1.775403e+06   
25%              12.063016          91.050000       2.441438e+06   
50%              14.939560         200.700000       2.840888e+06   
75%              18.868132         373.300000       3.763520e+06   
max              25.242391        1065.200000       4.031094e+06   

       windspeed_10m_mean  
count          175.000000  
mean            11.048944  
std              3.524397  
min              3.311957  
25%              9.788587  
50%             11.215385  
75%             13.246667  
max             21.804444

Above is a Tukey five number summary for the collected data. Though I strongly believe that the general overview is irrelevant at this point, the region-based summary has more value for the purposes of the project. So below is a Tukey five number summary for the North region in general.

In [11]:
# choose the North region for detailed summary
north_weather = weather_df[weather_df['region'] == 'Norte'] 
north_weather.describe()

temperature_2m_mean  precipitation_sum  sunshine_duration  \
count            25.000000           25.00000       2.500000e+01   
mean             13.281844          406.27600       2.893338e+06   
std               4.365668          282.68081       7.818858e+05   
min               7.940000           86.40000       1.775403e+06   
25%               8.982418          192.30000       2.173250e+06   
50%              12.184783          318.80000       2.574408e+06   
75%              16.034066          521.60000       3.654463e+06   
max              20.640217         1065.20000       3.881265e+06   

       windspeed_10m_mean  
count           25.000000  
mean             9.444451  
std              1.040243  
min              7.789130  
25%              8.479348  
50%              9.750000  
75%             10.013043  
max             11.070330

So here we can see all the statistical summary for each of the numerical data we have. All the other regions can be analyzed in the same manner and considering that the datasets will be merged, the analysis of one region should be considered more than enough for the EDA of the weather data in general.

---

As for the North region we have 25 samples (20 quarters for 5 years from 2019 to 2024 plus the first quarter of the 2025). Same amount of rows is for each region in the dataset. We can see that:

1. Temperature:
    - range is from 7.94 to 20.64 degrees, so we can see that the region had mild temperatures;
    - mean is 13.28 and median is 12.18, the difference is 1 degree, so we can assume that the data is a bit right-skewed;
    - the standard deviation is 4.37, which is not a lot, considering that the temperature range is in the mild spectrum, so we can suggest that most of the spread is from 8.91 to 17.55 degrees (13.28 ± 4.37);
    - the Q1(8.98) and the Q3(16.03) give us the understanding that 50% of the data is within these two values, which also almost corresponds to the variability-based conclusion;

2. Precipitation
    - the range is from 86.40 mm to 1065.20 mm per quarter, the contrast can be explained with the season change (summer vs winter);
    - the median (318.8 mm) is again below the mean (406 mm), implying a right-skewed distribution — a few unusually wet quarters raise the average;
    - the substantial variability is described by the quite large standard deviation (282.68 mm) compared to the mean (~70%), which indicates large seasonal flustuations;
    - the Q1(521, 6) and the Q3(192.3) imply that 50% of the data is within this range, implying high spread even among typical quarters;
3. Sunlight
    - The mean corresponds to roughly 9 hours of sunshine per day (2.89 million s ÷ (90 days × 3600 s/h) ≈ 8.9 h/day);
    - The min corresponds to ~5.5 h/day; the max to ~12 h/day which fits natural seasonal daylight variation (winter vs. summer);
    - The distribution is likely left-skewed (many lower-sunshine quarters, a few with extremely high sunshine). Overall high variability but no extreme outliers;
    - 25–75% range (≈ 2.17 – 3.65 million s) shows a big seasonal daylight swing;
4. Wind speed
    - the standard deviation is only ~11% of the mean, which implies that the wind speed is quite table;
    - slight left skew (median > mean), meaning most quarters are moderately windy, with a few calmer ones;
    - no major outliers or volatility, so the wond conditions are relatively constistent across quarters with minor seasonal variation.

In [13]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=north_weather, palette="Set2")
plt.title("Boxplots of Quarterly Weather Variables - North Region", fontsize=14)
plt.ylabel("Value")
plt.xlabel("Variable")
plt.show()


In [ ]:
cols = ["temperature_2m_mean", "precipitation_sum", "sunshine_duration", "windspeed_10m_mean"]
plot_titles = {
    "temperature_2m_mean": "Distribution of Mean Temperature (°C)",
    "precipitation_sum": "Distribution of Total Precipitation (mm)",
    "sunshine_duration": "Distribution of Sunshine Duration (hours)",
    "windspeed_10m_mean": "Distribution of Mean Windspeed (km/h)"
}
data = north_weather[cols]

sns.set(style="whitegrid", palette="Set2")
plt.figure(figsize=(12, 8))

for i, col in enumerate(cols, 1):
    plt.subplot(2, 2, i)
    if col == "sunshine_duration":
        sns.histplot(data[col] / 3600, bins=10, kde=True, color="gold", edgecolor="black")  # Convert seconds to hours
        plt.title(plot_titles[col])
        plt.xlabel(f"{col} (hours)")
    else:
        sns.histplot(data[col], bins=10, kde=True, color="skyblue", edgecolor="black")
        plt.title(plot_titles[col])
        plt.xlabel(col)
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()


In [17]:
north_weather.iloc[:, 2:].corr()

temperature_2m_mean  precipitation_sum  \
temperature_2m_mean             1.000000          -0.573648   
precipitation_sum              -0.573648           1.000000   
sunshine_duration               0.855465          -0.827341   
windspeed_10m_mean             -0.856414           0.810429   

                     sunshine_duration  windspeed_10m_mean  
temperature_2m_mean           0.855465           -0.856414  
precipitation_sum            -0.827341            0.810429  
sunshine_duration             1.000000           -0.896314  
windspeed_10m_mean           -0.896314            1.000000

From the correlation matrix above can see that
| Relationship                | Strength    | Direction | Interpretation         |
| --------------------------- | ----------- | --------- | ---------------------- |
| Temperature - Sunshine      | Strong      | Positive  | Warm = sunny           |
| Temperature - Precipitation | Moderate    | Negative  | Warm = dry, Cool = wet |
| Temperature - Windspeed     | Strong      | Negative  | Cool = windy           |
| Precipitation - Sunshine    | Strong      | Negative  | Rainy = cloudy         |
| Precipitation - Windspeed   | Strong      | Positive  | Rainy = windy          |
| Sunshine - Windspeed        | Very Strong | Negative  | Windy = less sun       |


In [ ]:
sns.pairplot(north_weather, diag_kind="kde")
plt.suptitle("Pairwise Relationships Between Weather Variables", y=1.02)
plt.show()